In [33]:
from xml.dom import minidom
import re
import pandas as pd
import sys
import pickle
import numpy as np
sys.__stdout__=sys.stdout
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    text=re.sub('\n',' ',text)
    #text=re.sub('-',' ',text)
    text=re.sub('[\'"]','',text)
    #text=re.sub('[()?.@*#&!,$]','',text)
    text=re.sub('[^a-z0-9]',' ',text)
    return TAG_RE.sub('', text)

def make_dictionary(topics,max_doc,root):
    index=0
    wordmap={}
    word_count=[]
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            if doc_count>=max_doc:
                break
            doc=doc.attributes['Body'].value
            doc=doc.lower()
            doc = remove_tags(doc)
            words=doc.split(' ')
            for word in words:
                if word in s_words:
                    continue
                if word not in wordmap:
                    wordmap[word]=index
                    word_count.append(int(1))
                    index=index+1
                elif word in wordmap:
                    ind=wordmap[word]
                    word_count[ind]=int(word_count[ind])+1
            doc_count=doc_count+1
    return wordmap,word_count
def vectorize_seq(topics,max_doc,wordmap,root):
    n_row=len(topics)*(max_doc)
    vectors=np.zeros((n_row,len(wordmap)),dtype=np.float64)
    vct_label=[]
    row=0
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            if doc_count>=max_doc or doc_count>=len(docs):
                break
            #vector = [0]*(len(wordmap)+2)
            doc=doc.attributes['Body'].value
            doc=doc.lower()
            doc = remove_tags(doc)
            words=doc.split(' ')
                     
            if len(words)<5:
                continue
            for word in words:
                if word in wordmap:
                    ind=wordmap[word]
                    vectors[row,ind]=vectors[row,ind]+1
            row=row+1
            vct_label.append(topic)
            word_list.append(words)
            doc_count=doc_count+1
    return vectors,vct_label


In [17]:
root='Dataset/Training/'
root_test='Dataset/Test/'
f=open('Stopwords.txt')
s_words=f.read()
s_words=s_words.split('\n')
s_words.append('')

topics=['3d_Printer','Coffee','Arduino','Astronomy','Biology']#,'Chess','Cooking','Law','Space','Windows_Phone','Wood_Working']
#topics=['Anime']


 
#wordmap=make_dictionary(topics,100,root)
#vectors=vectorize_seq(topics,100,wordmap,root)
word_list=[]
write_backup=1
if write_backup==1:
    wordmap,wc=make_dictionary(topics,500,root)
    vectors,vct_labels=vectorize_seq(topics,500,wordmap,root)
    pickle_out = open("wordmap_test.pickle","wb")
    pickle.dump(wordmap, pickle_out)
    pickle_out.close()
    pickle_out = open("vectors_test.pickle","wb")
    pickle.dump(vectors, pickle_out)
    pickle_out.close()

read_backup=0
if read_backup==1:
    print('Reading backup')
    pickle_read = open("wordmap_500.pickle","rb")
    wordmap=pickle.load(pickle_read)
    pickle_read.close()

    pickle_read = open("vectors_500.pickle","rb")
    vectors=pickle.load(pickle_read)
    pickle_read.close()

#words=list(wordmap.keys())
#freq=list(wordmap.values())
#word_df=pd.DataFrame({'words':words,'freq':freq})
#word_df=word_df.sort_values('freq',ascending=False)
    



3d_Printer
Coffee
Arduino
Astronomy
Biology
3d_Printer
Coffee
Arduino
Astronomy
Biology


In [18]:
print(len(wordmap))

20599


In [19]:
#print(word_list)

In [23]:
words=list(wordmap.keys())
freq=list(wordmap.values())
word_df=pd.DataFrame(words,index=list(wordmap.values()))
word_df=word_df.sort_index()
word_df['freq']=wc
word_df=word_df.sort_values('freq',ascending=False)
print(word_df[0:50])
#print(len(vectors))

                 0  freq
5706        coffee  1725
2639          code  1665
2421       arduino  1142
9              use   915
48            time   631
704     blockquote   628
449          water   517
2781           amp   474
97       questions   462
47           print   446
3020           pre   425
569            way   420
99       different   373
1166         stack   359
10167       serial   348
1167         imgur   335
6130      caffeine   334
319           good   331
90        filament   329
1065      question   329
14         printer   319
3077         board   317
1169           img   307
15              im   297
13              3d   297
198           long   292
29        printing   287
2310          data   283
6037      espresso   276
61     temperature   268
6035         beans   267
150           work   264
670            bit   263
13478         star   253
527          light   249
3109           pin   248
120          small   246
10196         nbsp   244
3             high   244


In [57]:
def eucl_dis(test_vect,vectors,vct_labels):
    h_dists=[]
    for row in range(len(vct_labels)):
        vect=vectors[row]
        dist=np.abs(vect-test_vect)
        dist=np.square(dist)
        dist=np.sum(dist)
        dist=np.sqrt(dist)
        label=vct_labels[row]
        h_dists.append([dist,label])
    return h_dists
def humm_dis(test_vector,vectors,vct_labels):
    h_dists=[]
    
    test_vect=np.array(test_vector>0,dtype=np.uint8)
    vects=np.array(vectors>0,dtype=np.uint8)
    #test_vect=test_vector
    #vects=vectors
    for row in range(len(vct_labels)):
        vect=vects[row]
        label=vct_labels[row]
        dist=np.abs(vect-test_vect)
        #dist=dist*[test_vect!=0]
        dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists
def cos_dis(test_vect,vectors,vct_labels):
    h_dists=[]
    
    vectors_bin= np.array(vectors>0,dtype=np.uint8)
    wc_vect=np.sum(vectors_bin,axis=0)
    no_docs=len(vectors)
    
    IDF=no_docs/wc_vect
    IDF=np.log(IDF)
    
    for row in range(len(vct_labels)):
        train_vect=vectors[row,:]
        
        train_label=vct_labels[row]
        
        train_total=np.sum(train_vect)
        test_total=np.sum(test_vect)
        
        TF_test=test_vect/test_total
        test_dis=TF_test*IDF
        
        temp=np.square(test_dis)
        temp=np.sum(temp)
        test_dis_mag=np.sqrt(temp)
        
        TF_train=train_vect/train_total
        train_dis=TF_train*IDF
        
        temp=np.square(train_dis)
        temp=np.sum(temp)
        train_dis_mag=np.sqrt(temp)
        
        X=np.sum(test_dis*train_dis)
        Y=train_dis_mag*test_dis_mag
        
        dist=X/Y
        h_dists.append([dist,train_label])
    return h_dists

In [58]:
def knn(nn,test_vect,vectors,vct_labels):
    print('getting h dist')
    #h_dists=eucl_dis(test_vect,vectors,vct_labels)
    #h_dists=humm_dis(test_vect,vectors,vct_labels)
    h_dists=cos_dis(test_vect,vectors,vct_labels)
    h_df=pd.DataFrame(h_dists,columns=['dist','label'])
    h_df=h_df.sort_values('dist',ascending=False)
    lb=h_df['label']
    lb=lb[:nn]
    lb=lb.mode()
    lb=lb.values[0]
    return(lb)
max_doc=60
test_vector,true_lbs=vectorize_seq(topics,max_doc,wordmap,root_test)
acc=0
n=len(topics)*max_doc
for i in range(n):
    true_lb=true_lbs[i]
    test_vect=test_vector[i,:]
    pred_lb=knn(3,test_vect,vectors,vct_labels)
    print(pred_lb,true_lb)
    if pred_lb==true_lb:
        acc=acc+1
        print(pred_lb,true_lb,acc,i)
accuracy=acc/(n)
print('accuray',accuracy)
#68.75% eucl 60 82.92
#53.33 humm 60  81.25
#87.08 cos 94.17

3d_Printer
Coffee
Arduino
Astronomy
Biology
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 1 0
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 1
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 2
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 4 3
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 5 4
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 6 5
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 7 6
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 8 7
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 9 8
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 10 9
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 11 10
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 12 11
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 13 12
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 14 13
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 15 14


KeyboardInterrupt: 

In [32]:
print(len(vectors))
print(len(vct_labels))
vects=np.array(vectors>0,dtype=np.uint8)
print(len(vects))

2500
2500
2500


In [ ]:
from matplotlib import pyplot as plt
vct=[vectors[12][:500]]
test=vectors[15][:498]
dis=eucl_dis(test,vct)
print(vct)
print(test)
print(dis[0][0])
x=range(0,500)
xt=range(0,498)
plt.plot(x,vct[0],xt,test)
plt.show

In [ ]:
def eucl_dis(test_vector,vectors):
    h_dists=[]
    for vector in vectors:
        vect=np.array(vector[:-2])
        vect[vect>1]=1
        label=vector[-1]
        test_vect=np.array(test_vector)
        dist=np.abs(vect-test_vect)
        #dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists
test_vector=vectors[0][:-2]
vecto=[vectors[0],vectors[1]]
h_dists=cos_dis(test_vector,vecto)
#h_df=pd.DataFrame(h_dists)
#h_df=h_df.sort_values(0)
print(h_dists)
#print((vectors[15]))

In [ ]:
for i in range(len(vectors)):
    vct=vectors[i][:-2]
    if np.sum(vct)==0:
        break
print(i,word_list[185])

In [109]:
raw='I am hasib\n,i "am" a student() so? @i do& my-duties! porp. and' 
#text=re.sub('\n',' ',raw)
text=raw.lower()
text=re.sub('[^a-z0-9]',' ',text)
#text=re.sub('-',' ',text)
#text=re.sub('[\'"]','',text)
#text=re.sub('[()?.@*#&!,$+]','',text)
#print(text.split(' '))